Downloading data in latlon format instead of llc. Is it easier? yeah i reckon so

In [ ]:
import os
import sys
sys.path.append("../ECCOv4-py")
import ecco_v4_py as ecco
from ecco_download import *
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
home = "~/Documents/Github/OTP/ECCO"
shortname = "ECCO_L4_TEMP_SALINITY_05DEG_MONTHLY_V4R4"

ecco_podaac_download(ShortName=shortname,
                     StartDate="2014-08-01", EndDate="2020-12-31", download_root_dir="data",
                     n_workers=6, force_redownload=True)

In [ ]:
ecco = xr.open_mfdataset(f"{home}/data/{shortname}/*.nc", parallel=True, 
                         data_vars="minimal", coords="minimal", compat="override")
osnap = xr.open_dataset("../OSNAP/OSNAP_Gridded_TSV_201408_202006_2023.nc")

# need to be careful to match depth, first.
ecco = ecco.sel(Z=-15.)
osnap = osnap.sel(DEPTH=15.)

In [ ]:
min_lat = osnap["LATITUDE"].min().values
max_lat = osnap["LATITUDE"].max().values
min_lon = osnap["LONGITUDE"].min().values
max_lon = osnap["LONGITUDE"].max().values

# osnap slice to reduce size.
ecco = ecco.sel(latitude=slice(min_lat, max_lat),
                longitude=slice(min_lon, max_lon))

# we'll take some lat/lon on the array.
lat = ecco["latitude"].median().values
lon = ecco["longitude"].median().values

ecco_subset = ecco.sel(latitude=lat, longitude=lon, method="nearest")
osnap_subset = osnap.sortby(["LATITUDE", "LONGITUDE"]).sel(LATITUDE=lat, LONGITUDE=lon, method="nearest")

In [ ]:
ecco_subset = ecco_subset.assign_coords(month=ecco_subset["time"].dt.month)
osnap_subset = osnap_subset.assign_coords(month=osnap_subset["TIME"].dt.month)

# Find the start and end of the common time range
common_start = max(ecco_subset["time"].min(), osnap_subset["TIME"].min())
common_end = min(ecco_subset["time"].max(), osnap_subset["TIME"].max())

# Subset the datasets to the common time range
ecco_subset = ecco_subset.sel(time=slice(common_start, common_end))
osnap_subset = osnap_subset.sel(TIME=slice(common_start, common_end))

# now the dates are only out by half a month (possibly just due to a convention difference) so when plotting I used the ecco time index.


print(ecco_subset["time"])
print(osnap_subset["TIME"])

merged = xr.merge([ecco_subset, osnap_subset], compat="override", join="outer")

In [ ]:
merged

In [ ]:
df = pd.DataFrame(columns=["time"])
t = merged["time"].values
T = merged["TIME"].values
print(len(t))
print(len(T))
e_theta = merged["THETA"].values
o_temp = merged["TEMP"].values
e_sal = merged["SALT"].values
o_sal = merged["SAL"].values

l = min(len(t), len(e_theta), len(o_temp), len(e_sal), len(o_sal))

df["time"] = t[:l]

df["ecco_theta"] = e_theta[:l]
df["osnap_temp"] = o_temp[:l]
df["ecco_sal"] = e_sal[:l]
df["osnap_sal"] = o_sal[:l]

df.set_index("time", inplace=True)


In [ ]:
print(df.head())

fig, ax = plt.subplots(2, 1, figsize=(12, 6))

ax[0].plot(df["ecco_theta"], label="ECCO", color="green", alpha=0.5, linestyle="-")
ax[0].plot(df["osnap_temp"], label="OSNAP", color="red", alpha=0.5, linestyle="--")
ax[0].legend()

ax[0].set_xlabel("date"); ax[0].set_ylabel("temp $\\degree$C")

ax[1].plot(df["ecco_sal"], label="ECCO", color="green", alpha=0.5, linestyle="-")
ax[1].plot(df["osnap_sal"], label="OSNAP", color="red", alpha=0.5, linestyle="--")
ax[1].legend()

ax[1].set_xlabel("date"); ax[1].set_ylabel("salinity")

plt.show()

print(np.corrcoef(df["ecco_theta"], df["osnap_temp"]))
print(np.corrcoef(df["ecco_sal"], df["osnap_sal"]))

plt.plot(df["ecco_theta"], df["osnap_temp"], "o", alpha=0.5)
plt.xlabel("ECCO")
plt.ylabel("OSNAP")
plt.show()

plt.plot(df["ecco_sal"], df["osnap_sal"], "o", alpha=0.5)
plt.xlabel("ECCO")
plt.ylabel("OSNAP")
plt.show()